# License Plate OCR Evaluation

This notebook analyzes the performance of OCR for license plate recognition by comparing predicted values with ground truth.


In [ ]:
# Import required libraries
import yaml
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

# Set display options
pd.set_option("display.max_rows", 100)
sns.set_theme(style="whitegrid")
plt.rcParams["font.family"] = "Iosevka Aile"

## Load the Data

We'll load both the ground truth and OCR results from YAML files.


In [ ]:
# Load OCR results and ground truth
with open("license_plate_results_with_preprocessing.yaml", "r") as f:
    ocr_results = yaml.unsafe_load(f)

with open("dataset/val_plate_numbers.yaml", "r") as f:
    ground_truth = yaml.safe_load(f)

with open("dataset/val_plate_type.yaml", "r") as f:
    vehicle_type = yaml.safe_load(f)

# Check the data
print(f"Number of OCR results: {len(ocr_results)}")
print(f"Number of ground truth entries: {len(ground_truth)}")
print(f"Number of vehicle type entries: {len(vehicle_type)}")

# Preview a few examples
print("\nOCR Results (Sample):")
for i, (key, value) in enumerate(list(ocr_results.items())[:5]):
    print(f"{key}: {value}")

print("\nGround Truth (Sample):")
for i, (key, value) in enumerate(list(ground_truth.items())[:5]):
    print(f"{key}: {value}")

print("\nVehicle Type (Sample):")
for i, (key, value) in enumerate(list(vehicle_type.items())[:5]):
    print(f"{key}: {value}")

## Prepare Data for Analysis

Let's create a DataFrame to compare OCR results with ground truth.


In [ ]:
# Create a dataset for analysis
data = []

for image_id, ocr_result in ocr_results.items():
    # Get OCR result components
    is_detected = ocr_result[0]
    detected_text = ocr_result[1]

    # Get ground truth if it exists
    true_text = ground_truth.get(image_id)

    # Calculate if the OCR result matches ground truth
    is_correct = False
    if true_text is not None and detected_text is not None:
        is_correct = detected_text == true_text

    data.append(
        {
            "image_id": image_id,
            "is_detected": is_detected,
            "detected_text": detected_text,
            "ground_truth": true_text,
            "is_correct": is_correct,
            "vehicle_type": int(vehicle_type[image_id]),
        }
    )

for image_id, truth in ground_truth.items():
    if image_id in [row["image_id"] for row in data]:
        continue
    data.append(
        {
            "image_id": image_id,
            "is_detected": False,
            "detected_text": "",
            "ground_truth": truth,
            "is_correct": False,
            "vehicle_type": int(vehicle_type[image_id]),
        }
    )

# Convert to DataFrame
df = pd.DataFrame(data)


# Filter out entries where ground truth is None (if needed)
df_valid = df[df["ground_truth"].notna()]

# Preview the dataframe
df_valid.head()

## Calculate Overall Accuracy Metrics


In [ ]:
# Calculate overall accuracy metrics
total_entries = len(df_valid)
correctly_detected_count = df_valid[df_valid["is_correct"]].shape[0]
incorrectly_detected_count = df_valid[
    (df_valid["detected_text"].str.len() == 8) & ~df_valid["is_correct"]
].shape[0]
missed_detection_count = df_valid[~(df_valid["detected_text"].str.len() == 8)].shape[0]

# Calculate percentages
detection_rate = (
    (correctly_detected_count + incorrectly_detected_count) / total_entries * 100
)
accuracy_rate = correctly_detected_count / total_entries * 100
error_rate = incorrectly_detected_count / total_entries * 100
miss_rate = missed_detection_count / total_entries * 100

print(f"Total license plates: {total_entries}")
print(f"Detection rate: {detection_rate:.2f}%")
print(f"Accuracy rate: {accuracy_rate:.2f}%")
print(f"Error rate: {error_rate:.2f}%")
print(f"Miss rate: {miss_rate:.2f}%")

# Calculate accuracy among detected plates
detected_plates = df_valid[df_valid["detected_text"].str.len() == 8]
detected_accuracy = detected_plates["is_correct"].mean() * 100
print(f"Accuracy among detected plates: {detected_accuracy:.2f}%")

## Visualize Results


In [ ]:
correctly_detected_car = df_valid[
    df_valid["is_correct"] & (df_valid["vehicle_type"] == 1)
].shape[0]
incorrectly_detected_car = df_valid[
    (df_valid["detected_text"].str.len() == 8)
    & ~df_valid["is_correct"]
    & (df_valid["vehicle_type"] == 1)
].shape[0]
missed_detection_car = df_valid[
    ~(df_valid["detected_text"].str.len() == 8) & (df_valid["vehicle_type"] == 1)
].shape[0]

correctly_detected_truck = df_valid[
    df_valid["is_correct"] & (df_valid["vehicle_type"] == 2)
].shape[0]
incorrectly_detected_truck = df_valid[
    (df_valid["detected_text"].str.len() == 8)
    & ~df_valid["is_correct"]
    & (df_valid["vehicle_type"] == 2)
].shape[0]
missed_detection_truck = df_valid[
    ~(df_valid["detected_text"].str.len() == 8) & (df_valid["vehicle_type"] == 2)
].shape[0]

correctly_detected_motorbike = df_valid[
    df_valid["is_correct"] & (df_valid["vehicle_type"] == 3)
].shape[0]
incorrectly_detected_motorbike = df_valid[
    (df_valid["detected_text"].str.len() == 8)
    & ~df_valid["is_correct"]
    & (df_valid["vehicle_type"] == 3)
].shape[0]
missed_detection_motorbike = df_valid[
    ~(df_valid["detected_text"].str.len() == 8) & (df_valid["vehicle_type"] == 3)
].shape[0]

labels = ["Правильно", "Неправильно", "Пропущено"]
colors = ["#4CAF50", "#FFC107", "#F44336"]
explode = (0.05, 0, 0)

sizes_car = [correctly_detected_car, incorrectly_detected_car, missed_detection_car]
sizes_truck = [
    correctly_detected_truck,
    incorrectly_detected_truck,
    missed_detection_truck,
]
sizes_motorbike = [
    correctly_detected_motorbike,
    incorrectly_detected_motorbike,
    missed_detection_motorbike,
]

sizes_overall = [
    correctly_detected_count,
    incorrectly_detected_count,
    missed_detection_count,
]

# Конвертація см у дюйми
cm_to_inch = 1 / 2.54
fig_width = 16 * cm_to_inch  # 16 см ≈ 6.3 дюймів
fig_height = 6 * cm_to_inch  # 8 см ≈ 3.15 дюймів

fig, axs = plt.subplots(1, 4, figsize=(fig_width, fig_height), dpi=300)

# Автомобілі
wedges_car, texts_car, autotexts_car = axs[0].pie(
    sizes_car,
    colors=colors,
    explode=explode,
    autopct="%1.1f%%",
    startangle=140,
    shadow=True,
    labels=None,
)
axs[0].set_title("Автомобілі", fontsize=12)
axs[0].axis("equal")

# Вантажівки
wedges_truck, texts_truck, autotexts_truck = axs[1].pie(
    sizes_truck,
    colors=colors,
    explode=explode,
    autopct="%1.1f%%",
    startangle=140,
    shadow=True,
    labels=None,
)
axs[1].set_title("Вантажівки", fontsize=12)
axs[1].axis("equal")

# Мотоцикли
wedges_motorbike, texts_motorbike, autotexts_motorbike = axs[2].pie(
    sizes_motorbike,
    colors=colors,
    explode=explode,
    autopct="%1.1f%%",
    startangle=140,
    shadow=True,
    labels=None,
)
axs[2].set_title("Мотоцикли", fontsize=12)
axs[2].axis("equal")

# Загальні результати
wedges_overall, texts_overall, autotexts_overall = axs[3].pie(
    sizes_overall,
    colors=colors,
    explode=explode,
    autopct="%1.1f%%",
    startangle=140,
    shadow=True,
    labels=None,
)
axs[3].set_title("Загалом", fontsize=12)
axs[3].axis("equal")

legend = fig.legend(
    wedges_car,
    labels,
    loc="upper center",
    bbox_to_anchor=(0.5, 0.05),
    ncol=3,
    fontsize=10,
)

# Make the suptitle an artist we can pass to savefig so it won't be cropped
supt = fig.suptitle("Розпізнавання номерних знаків за типом ТЗ", fontsize=12, y=0.98)

# Reserve top space so tight_layout doesn't overlap the suptitle
fig.tight_layout(rect=[0, 0.03, 1, 0.95], pad=0.02)

# Slight manual adjustments for margins
fig.subplots_adjust(left=0.01, right=0.99, bottom=0.02, wspace=0.02)

fig.savefig(
    "temp/figs/accuracy_pie_chart_by_vehicle_type.svg",
    format="svg",
    bbox_inches="tight",
    pad_inches=0.01,
    bbox_extra_artists=(legend, supt),
)
plt.show()


## Character-Level Analysis


In [ ]:
import Levenshtein

df_ocr_eval = df_valid[["detected_text", "ground_truth", "vehicle_type"]].copy()

df_ocr_eval["distance"] = df_ocr_eval.apply(
    lambda x: Levenshtein.distance(x["detected_text"], x["ground_truth"]),
    axis=1,
)

df_ocr_eval.head()

In [ ]:
type_error_counts = (
    df_ocr_eval.groupby(["vehicle_type", "distance"])
    .size()
    .unstack(fill_value=0)
    .reset_index()
    .melt(id_vars="vehicle_type", var_name="distance", value_name="count")
)

wrong_error_counts_by_type = (
    df_ocr_eval[
        (df_ocr_eval["detected_text"].str.len() == 8) & (df_ocr_eval["distance"] > 0)
    ]
    .groupby(["vehicle_type", "distance"])
    .size()
    .unstack(fill_value=0)
    .reset_index()
    .melt(id_vars="vehicle_type", var_name="distance", value_name="count")
)

missed_error_counts = (
    df_ocr_eval[df_ocr_eval["detected_text"].str.len() != 8]
    .groupby(["vehicle_type", "distance"])
    .size()
    .unstack(fill_value=0)
    .reset_index()
    .melt(id_vars="vehicle_type", var_name="distance", value_name="count")
)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


# Створимо допоміжну функцію для малювання накопичувального графіка
def draw_stacked_bar(data, ax, title, colors):
    # Повертаємо дані з "long" формату (після melt) назад у "wide" для зручного стакування
    pivot_df = data.pivot(
        index="distance", columns="vehicle_type", values="count"
    ).fillna(0)

    # Малюємо stacked bar
    pivot_df.plot(kind="bar", stacked=True, ax=ax, color=colors, width=0.8)

    ax.set_title(title, fontsize=10)
    ax.set_xlabel("Відстань Левенштейна", fontsize=10)
    ax.set_ylabel("Кількість", fontsize=10)
    ax.legend().remove()  # Прибираємо локальні легенди, залишимо одну загальну


fig, axs = plt.subplots(3, 1, figsize=(6.7, 8), dpi=300)

# Визначаємо кольори відповідно до вашої палітри Set2
colors = ["#66c2a5", "#fc8d62", "#8da0cb"]

# Підготовка даних та малювання (приклади для кожного сабплота)
# 1. Загалом
draw_stacked_bar(type_error_counts, axs[0], "Загалом", colors)

# 2. Неправильні
draw_stacked_bar(wrong_error_counts_by_type, axs[1], "Неправильні", colors)

# 3. Пропущені
draw_stacked_bar(missed_error_counts, axs[2], "Пропущені", colors)

# Налаштування загальної легенди
handles, labels = axs[0].get_legend_handles_labels()
# Якщо потрібно замінити англійські назви на українські:
labels = ["Автомобілі", "Вантажівки", "Мотоцикли"]

fig.legend(
    handles,
    labels,
    loc="center left",
    bbox_to_anchor=(1.0, 0.5),
    title="Тип ТЗ",
    fontsize=10,
    title_fontsize=12,
)

plt.suptitle("Розподіл помилок OCR за типом ТЗ", fontsize=14, y=0.98)
plt.tight_layout()

plt.savefig(
    "temp/figs/ocr_error_distribution_by_vehicle_type.svg",
    format="svg",
    bbox_inches="tight",
)
plt.show()

In [ ]:
def analyze_character_confusion(detected, ground_truth):
    if detected is None or ground_truth is None:
        return {}

    errors = []
    min_len = min(len(detected), len(ground_truth))

    for i in range(min_len):
        if detected[i] != ground_truth[i]:
            errors.append((ground_truth[i], detected[i]))

    return Counter(errors)


# Collect all character-level errors
all_char_errors = Counter()
for _, row in df_valid[df_valid["is_detected"] & ~df_valid["is_correct"]].iterrows():
    errors = analyze_character_confusion(row["detected_text"], row["ground_truth"])
    all_char_errors.update(errors)

for _, row in df_valid[~df_valid["is_detected"]].iterrows():
    if not len(row["detected_text"]) == 8:
        continue
    errors = analyze_character_confusion(row["detected_text"], row["ground_truth"])
    all_char_errors.update(errors)

# Display the most common errors
print("Most common character-level errors (ground_truth → detected):")
for (true_char, detected_char), count in all_char_errors.most_common(15):
    print(f"'{true_char}' → '{detected_char}': {count} times")

## Confusion Matrix for Character Recognition


In [ ]:
def create_confusion_matrix(errors):
    # Get unique characters
    all_chars = set()
    for true_char, pred_char in errors.keys():
        if true_char:
            all_chars.add(true_char)
        if pred_char:
            all_chars.add(pred_char)

    all_chars = sorted(list(all_chars))
    n = len(all_chars)

    # Create mapping for char to index
    char_to_idx = {char: idx for idx, char in enumerate(all_chars)}

    # Initialize confusion matrix
    conf_matrix = np.zeros((n, n))

    # Fill confusion matrix
    for (true_char, pred_char), count in errors.items():
        if true_char and pred_char:  # Skip insertions and deletions
            true_idx = char_to_idx[true_char]
            pred_idx = char_to_idx[pred_char]
            conf_matrix[pred_idx, true_idx] += count

    return conf_matrix, all_chars


# Create character confusion matrix
char_errors = {
    k: v for k, v in all_char_errors.items() if k[0] and k[1]
}  # Filter out insertions/deletions
if char_errors:
    conf_matrix, all_chars = create_confusion_matrix(char_errors)

    # Plot confusion matrix
    plt.figure(figsize=(6.3, 6.3), dpi=300)
    sns.heatmap(
        conf_matrix,
        annot=True,
        fmt="g",
        cmap="Blues",
        xticklabels=all_chars,
        yticklabels=all_chars,
        annot_kws={"size": 10},
    )
    plt.title("Матриця плутатнини", fontsize=14)
    plt.xlabel("Істині символи")
    plt.ylabel("Розпізнані символи")
    plt.tight_layout()
    plt.savefig(
        "temp/figs/character_confusion_matrix.svg",
        format="svg",
        bbox_inches="tight",
    )
    plt.show()

In [ ]:
# Normalize by column (true characters)
col_sums = conf_matrix.sum(axis=0)
# Avoid division by zero for columns that sum to 0
col_sums[col_sums == 0] = 1
normalized_conf_matrix = conf_matrix / col_sums

# Create character confusion matrix
char_errors = {
    k: v for k, v in all_char_errors.items() if k[0] and k[1]
}  # Filter out insertions/deletions
if char_errors:
    # Plot confusion matrix
    plt.figure(figsize=(6.3, 6.3), dpi=300)
    sns.heatmap(
        normalized_conf_matrix,
        annot=True,
        fmt=".1f",
        cmap="Blues",
        xticklabels=all_chars,
        yticklabels=all_chars,
        vmin=0,
        vmax=1,
        annot_kws={"size": 5},
    )
    plt.title("Нормалізована матриця плутатнини", fontsize=14)
    plt.xlabel("Істині символи")
    plt.ylabel("Розпізнані символи")
    plt.tight_layout()
    plt.savefig(
        "temp/figs/character_confusion_matrix_normalized.svg",
        format="svg",
        bbox_inches="tight",
    )
    plt.show()